Plotting SuperMuSR data
***********************
The way to work with this data is by changing the values in the first cell of this notebook, you shouldn't need to change the rest, a plot should appear in a pop out window using matplotlib. If sample_rate is found in the data then it will use the sample_rate to calculate sample time, otherwise it will just display the value of the sample number.

Values:
- filename -> Needs to be a path to a valid .h5 file either absolute or relative to the location of this .ipynb file.
- frame -> The instance frame that needs to be plotted (Optional, but only if you give a timestamp)
- timestamp -> If the frame is not given, this is required, otherwise it will not be used. It expects ISO8601 UTC, https://www.iso.org/iso-8601-date-and-time-format.html
- channels -> It's a list of channels to plot, it is likely these are detectors. There are multiples of 8 channels present in most of the files tested with this.

In [1]:
# These values are set here and used further down
filename = "trace-Tue 26 Jul 10-57-38 BST 2022.h5"
frame = 400 # Expected if plot_frame is True, alternatively use a timestamp
timestamp = "2022-07-26T10:09:12.890" # Expected in ISO8601 UTC (Note: the data takes into consideration nanoseconds)
channels = [0, 1, 2, 3, 4, 5, 6, 7] # A list of channels to plot, first value is 0, last value is size-1

In [2]:
import h5py
import dateutil.parser as dp

data = h5py.File(filename, 'r')
detector_data = data['detector_data']
seconds_list = data['frame_timestamp']['seconds']
nanoseconds_list = data['frame_timestamp']['nanoseconds']
frame_list = data['frame_start_index']
if "/sample_rate" in data:
    sample_rate = data['sample_rate'][()]
else:
    sample_rate = None

# Ensure that the frame is set, and if not then try find it from the timestamp
if frame is None:
    if timestamp is None:
        raise Exception("If frame is None, timestamp needs to be given for the frame otherwise we can't determine the frame to output")
    else:
        timestamp_seconds = dp.parse(timestamp).timestamp() # Convert timestamp to seconds + nanoseconds
        # Work out the frame from the timestamp
        for index in range(0, len(seconds_list)-1):
            first = seconds_list[index] + nanoseconds_list[index] * 1e-9
            second = seconds_list[index+1] + nanoseconds_list[index+1] * 1e-9
            if first < timestamp_seconds <= second:
                frame = index
                break

if frame is None:
    raise Exception("Frame was not set, and the frame could not be found from input data")

In [3]:
# Grab the frame's data
lower_limit = frame_list[frame]
upper_limit = frame_list[frame+1]
frame_data_list = []
for channel in channels:
    frame_data_list.append(detector_data[channel][lower_limit:upper_limit])

In [4]:
# Generate the x-axis from the frame and the sample_rate
# The x axis is sample_time, sample time (s) = sample number (1) / sample rate (1/s
if sample_rate is not None:
    x_axis_time = []
    for index in range(0, len(frame_data_list[0])):
        x_axis_time.append(index / sample_rate)
else:
    x_axis_time = None

In [5]:
%matplotlib qt
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

for frame, channel in zip(frame_data_list, channels):
    if x_axis_time is not None:
        line = ax.plot(x_axis_time, frame)
    else:
        line = ax.plot(frame)
    line[0].set_label(f"Channel {channel}")
legend = ax.legend()
if legend:
    legend.set_draggable(True)
ax.set_title(timestamp)
ax.set_xlabel("sample")
ax.set_ylabel("intensity")


Text(0, 0.5, 'intensity')